<a href="https://colab.research.google.com/github/agatalango/real_garbage_collector/blob/main/Garbage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project name: *Computer vision model recognizing different types of garbage using openCV.*

Team members: **Katarzyna Iwaszkiewicz, Agata Lango, Dominik Zięba**

Dataset: https://www.kaggle.com/datasets/mostafaabla/garbage-classification/data

GIT repo: https://github.com/agatalango/real_garbage_collector



# 1. Import required libraries

In [14]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, shutil, random
import cv2
import sklearn
import tensorflow as tf
from tensorflow import keras
from PIL import Image as im
from glob import glob
from sklearn.model_selection import train_test_split
import keras
#from tf.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from PIL import Image as im
import cv2

# 2. Create a dictionary from 12 classes

In [15]:
root='https://github.com/agatalango/real_garbage_collector/tree/513457570c0234356449cb1d61fc5451994e7149/garbage_classification'
data={}
for i in os.listdir(root):
    for j in os.walk(root+i):
        for k in j[2]:
            data[root+i+'/'+k]=i
data=pd.DataFrame(data.items(),columns=['path','class_'])
data['class_'].value_counts().plot(kind='bar')
plt.title('Data 12 Class (VERY UNBALANCED)')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'https://github.com/agatalango/real_garbage_collector/tree/513457570c0234356449cb1d61fc5451994e7149/garbage_classification'

In [ ]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3

base_path = "https://github.com/agatalango/real_garbage_collector/tree/main/garbage_classification"

categories = {0: 'paper', 1: 'cardboard', 2: 'plastic', 3: 'metal', 4: 'trash', 5: 'glass'}

print('defining constants successful!')

#Add class name prefix to filename. So for example "/paper104.jpg" become "paper/paper104.jpg"
def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[:re.search("\d",x).start()] + '/' + x)
    return df

base_path = "/garbage-classification/garbage_classification/"
# Creating a dictionary from 12 classes
categories={}
i=0
for dirname, _, filenames in os.walk('https://github.com/agatalango/real_garbage_collector/tree/513457570c0234356449cb1d61fc5451994e7149/garbage_classification'):
    for filename in filenames:
        categories[i] = dirname.split('/')[-1]
        break
        print(os.path.join(dirname, filename))
    i += 1
print(categories)
print('defining constants successful!')

defining constants successful!
{}
defining constants successful!


In [ ]:
#Add class name prefix to filename. So for example "/paper104.jpg" become "paper/paper104.jpg"
#def add_class_name_prefix(df, col_name):
#    df[col_name] = df[col_name].apply(lambda x: x[:re.search("\d",x).start()] + '/' + x)
#    return df

# 3. Create dataframe

In [ ]:
linkcode
import os, shutil, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_dir = os.path.join("/kaggle/working/", "train")
test_dir = os.path.join("/kaggle/working/", "test")
origin_dir = os.path.join("/kaggle", "input", "garbage-classification",  "garbage_classification")
categories = os.listdir(origin_dir)

for category in categories:
    category_src_path = os.path.join(origin_dir, category)
    category_dst_path_train = os.path.join(train_dir, category)
    category_dst_path_test = os.path.join(test_dir, category)
    os.makedirs(category_dst_path_train, exist_ok = True)
    os.makedirs(category_dst_path_test, exist_ok = True)

    file_names = os.listdir(category_src_path)
    random.shuffle(file_names)
    length = len(file_names)
    train_file_names = file_names[:int(length*0.8)]
    test_file_names = file_names[int(length*0.8):]

    for name in train_file_names:
        src_path = os.path.join(category_src_path, name)
        dst_path = os.path.join(category_dst_path_train, name)
        shutil.copyfile(src = src_path,
                        dst = dst_path)

    for name in test_file_names:
        src_path = os.path.join(category_src_path, name)
        dst_path = os.path.join(category_dst_path_test, name)
        shutil.copyfile(src = src_path,
                        dst = dst_path)

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                             validation_split = 0.2)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

train_data = train_gen.flow_from_directory("/kaggle/working/train",
                                            subset = "training", seed = 42, target_size = (299, 299))
val_data = train_gen.flow_from_directory("/kaggle/working/train",
                                            subset = "validation", seed = 42, target_size = (299, 299))
test_data = test_gen.flow_from_directory("/kaggle/working/test", seed = 42, target_size = (299, 299))

In [ ]:
class_counts = {}

train_path = os.path.join("/kaggle/working/train")
for cls in os.listdir(train_path):
    class_path = os.path.join(train_path, cls)
    class_counts[cls] = len(os.listdir(class_path))

class_counts = dict(sorted(class_counts.items(), key=lambda x:x[1], reverse = True))
class_counts

In [ ]:
g = sns.barplot(x = list(class_counts.values()), y = list(class_counts.keys()))

In [ ]:
class_names = list(train_data.class_indices.keys())
class_names

In [ ]:
########### list conatining all the filenames in the dataset
filenames_list = []
# list to store the corresponding category, note that each folder of the dataset has one class of data
categories_list = []

for category in categories:
    filenames = os.listdir(base_path + categories[category])

    filenames_list = filenames_list  +filenames
    categories_list = categories_list + [category] * len(filenames)


df = pd.DataFrame({
    'filename': filenames_list,
    'category': categories_list
})

df = add_class_name_prefix(df, 'filename')

df = df.sample(frac=1).reset_index(drop=True)

print('number of elements = ' , len(df))



NameError: name 'categories' is not defined

In [ ]:
df.head()

In [ ]:
# see sample image, you can run the same cell again to get a different image
random_row = random.randint(0, len(df)-1)
sample = df.iloc[random_row]
randomimage = image.load_img(base_path +sample['filename'])

# 4. Visualize the categories distribution


In [ ]:
df_visualization = df.copy()
# Change the catgegories from numbers to names
df_visualization['category'] = df_visualization['category'].apply(lambda x:categories[x] )

df_visualization['category'].value_counts().plot.bar(x = 'count', y = 'category' )

plt.xlabel("Garbage Classes", labelpad=14)
plt.ylabel("Images Count", labelpad=14)
plt.title("Count of images per class", y=1.02);

# 5. Split dataset into train, test and validation sets: 80% train set, 10% cross_validation set, and 10% test set (TBC)

In [ ]:
#Change the categories from numbers to names
df["category"] = df["category"].replace(categories)

# We first split the data into two sets and then split the validate_df to two sets
train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.3, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

print('train size = ', total_validate , 'validate size = ', total_validate, 'test size = ', test_df.shape[0])

# 6. Create model

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import keras.applications.xception as xception

xception_layer = xception.Xception(include_top = False, input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS),
                       weights = '../input/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')

# We don't want to train the imported weights
xception_layer.trainable = False


model = Sequential()
model.add(keras.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))

#create a custom layer to apply the preprocessing
def xception_preprocessing(img):
  return xception.preprocess_input(img)

model.add(Lambda(xception_preprocessing))

model.add(xception_layer)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(len(categories), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.summary()

In [ ]:
#Train model
...

EPOCHS = 20
history = model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Train the model

In [ ]:
batch_size=64

train_datagen = image.ImageDataGenerator(

    ###  Augmentation Start  ###

    rotation_range=30,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip = True,
    width_shift_range=0.2,
    height_shift_range=0.2

    ##  Augmentation End  ###
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    base_path,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
validation_datagen = image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    base_path,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
EPOCHS = 7
history = model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    #callbacks=callbacks
)

In [ ]:
model.save_weights("model12.h5")

# 7. Visualize training process

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_yticks(np.arange(0, 0.7, 0.1))
ax1.legend()

ax2.plot(history.history['categorical_accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_categorical_accuracy'], color='r',label="Validation accuracy")
ax2.legend()

legend = plt.legend(loc='best')
plt.tight_layout()
plt.show()

# 8. Validation of training process

# 9. Save model hdf5